# Importando as bibliotecas necessárias

In [2]:
import cv2
import os
import glob
import numpy as np

# Obtenção das fotos do tabuleiro para calibração das cameras e consequentemente obtenção dos POI

In [13]:
# Define the key to capture the images
capture_key = ord('c')  # Change it to the desired key

# Define the camera indices (0 and 1 for the first and second cameras)
camera_indices = [1, 2]


# Create directories for saving images
image_dirs = ['camera1_images', 'camera2_images']
for dir_name in image_dirs:
    os.makedirs(dir_name, exist_ok=True)
    os.makedirs(dir_name + '_calibrated', exist_ok=True)

# Initialize the cameras
cameras = []
for index in camera_indices:
    camera = cv2.VideoCapture(index)
    cameras.append(camera)

counter = 0
# Start capturing images
while True:
    # Read frames from each camera
    frames = []
    for camera in cameras:
        ret, frame = camera.read()
        frames.append(frame)

    # Display the frames from each camera
    for i, frame in enumerate(frames):
        cv2.imshow(f'Camera {i+1}', frame)

    # Check if the capture key is pressed
    key = cv2.waitKey(1)
    if key == capture_key:
        counter+=1
        # Save the images to respective folders
        for i, frame in enumerate(frames):
            image_path = os.path.join(image_dirs[i], f'image_{i+1}_{counter}.jpg')
            cv2.imwrite(image_path, frame)
            print(f'Image saved: {image_path}')
    # Check if the 'q' key is pressed to quit the program
    if key == ord('q'):
        break

# Release the cameras and close windows
for camera in cameras:
    camera.release()
cv2.destroyAllWindows()

In [110]:
def same_pict():
    # Define the key to capture the images
    capture_key = ord('c')  # Change it to the desired key

    # Define the camera indices (0 and 1 for the first and second cameras)
    camera_indices = [1, 2]


    # Create directories for saving images
    image_dirs = 'camera_1_2_frames'
    os.makedirs(image_dirs, exist_ok=True)

    # Initialize the cameras
    cameras = []
    for index in camera_indices:
        camera = cv2.VideoCapture(index)
        cameras.append(camera)

    counter = 0
    # Start capturing images
    while True:
        # Read frames from each camera
        frames = []
        for camera in cameras:
            ret, frame = camera.read()
            frames.append(frame)

        # Display the frames from each camera
        for i, frame in enumerate(frames):
            cv2.imshow(f'Camera {i+1}', frame)

        # Check if the capture key is pressed
        key = cv2.waitKey(1)
        if key == capture_key:
            counter+=1
            # Save the images to respective folders
            for i, frame in enumerate(frames):
                image_path = os.path.join(image_dirs, f'image_{i+1}_{counter}.jpg')
                cv2.imwrite(image_path, frame)
                print(f'Image saved: {image_path}')
        # Check if the 'q' key is pressed to quit the program
        if key == ord('q'):
            break

    # Release the cameras and close windows
    for camera in cameras:
        camera.release()
    cv2.destroyAllWindows()
same_pict()

Image saved: camera_1_2_frames\image_1_1.jpg
Image saved: camera_1_2_frames\image_2_1.jpg
Image saved: camera_1_2_frames\image_1_2.jpg
Image saved: camera_1_2_frames\image_2_2.jpg
Image saved: camera_1_2_frames\image_1_3.jpg
Image saved: camera_1_2_frames\image_2_3.jpg
Image saved: camera_1_2_frames\image_1_4.jpg
Image saved: camera_1_2_frames\image_2_4.jpg
Image saved: camera_1_2_frames\image_1_5.jpg
Image saved: camera_1_2_frames\image_2_5.jpg
Image saved: camera_1_2_frames\image_1_6.jpg
Image saved: camera_1_2_frames\image_2_6.jpg
Image saved: camera_1_2_frames\image_1_7.jpg
Image saved: camera_1_2_frames\image_2_7.jpg
Image saved: camera_1_2_frames\image_1_8.jpg
Image saved: camera_1_2_frames\image_2_8.jpg
Image saved: camera_1_2_frames\image_1_9.jpg
Image saved: camera_1_2_frames\image_2_9.jpg
Image saved: camera_1_2_frames\image_1_10.jpg
Image saved: camera_1_2_frames\image_2_10.jpg


# Obtenção dos POI

In [6]:
def draw_chess_write(camera):
  # termination criteria
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
  # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
  world_scaling = 2.8
  objp = np.zeros((6*8,3), np.float32)
  objp[:,:2] = np.mgrid[0:8,0:6].T.reshape(-1,2)
  objp = world_scaling* objp
  # Arrays to store object points and image points from all the images.
  objpoints = [] # 3d point in real world space
  imgpoints = [] # 2d points in image plane.
  images = glob.glob(f'./camera{camera}_images/*.jpg')
  i=1
  for fname in images:
    
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (8,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        name = i
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (8,6), corners2, ret)
        cv2.imshow('img',img)
        cv2.imwrite(f'./camera{camera}_images_calibrated/calib_' + str(name) + '_calib.jpg',img)
        cv2.waitKey(500)
        i+=1
        
  cv2.destroyAllWindows()
  return objpoints, imgpoints, gray

def stereo_calibrate(mtx0, dist0, mtx1, dist1, frames_prefix_c0, frames_prefix_c1):
    #read the synched frames
    c0_images_names = sorted(glob.glob(frames_prefix_c0))
    c1_images_names = sorted(glob.glob(frames_prefix_c1))

    #open images
    c0_images = [cv2.imread(imname, 1) for imname in c0_images_names]
    c1_images = [cv2.imread(imname, 1) for imname in c1_images_names]

    #change this if stereo calibration not good.
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)

    #calibration pattern settings
    rows = 8
    columns = 6
    world_scaling = 2.8

    #coordinates of squares in the checkerboard world space
    objp = np.zeros((rows*columns,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:columns].T.reshape(-1,2)
    objp = world_scaling* objp

    #frame dimensions. Frames should be the same size.
    width = c0_images[0].shape[1]
    height = c0_images[0].shape[0]

    #Pixel coordinates of checkerboards
    imgpoints_left = [] # 2d points in image plane.
    imgpoints_right = []

    #coordinates of the checkerboard in checkerboard world space.
    objpoints = [] # 3d point in real world space

    for frame0, frame1 in zip(c0_images, c1_images):
        gray1 = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        c_ret1, corners1 = cv2.findChessboardCorners(gray1, (rows, columns), None)
        c_ret2, corners2 = cv2.findChessboardCorners(gray2, (rows, columns), None)

        if c_ret1 == True and c_ret2 == True:

            corners1 = cv2.cornerSubPix(gray1, corners1, (11, 11), (-1, -1), criteria)
            corners2 = cv2.cornerSubPix(gray2, corners2, (11, 11), (-1, -1), criteria)

            p0_c1 = corners1[0,0].astype(np.int32)
            p0_c2 = corners2[0,0].astype(np.int32)

            cv2.putText(frame0, 'O', (p0_c1[0], p0_c1[1]), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 1)
            cv2.drawChessboardCorners(frame0, (rows,columns), corners1, c_ret1)
            cv2.imshow('img', frame0)

            cv2.putText(frame1, 'O', (p0_c2[0], p0_c2[1]), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 1)
            cv2.drawChessboardCorners(frame1, (rows,columns), corners2, c_ret2)
            cv2.imshow('img2', frame1)
            k = cv2.waitKey(0)

            if k & 0xFF == ord('s'):
                print('skipping')
                continue

            objpoints.append(objp)
            imgpoints_left.append(corners1)
            imgpoints_right.append(corners2)

    stereocalibration_flags = cv2.CALIB_FIX_INTRINSIC
    ret, CM1, dist0, CM2, dist1, R, T, E, F = cv2.stereoCalibrate(objpoints, imgpoints_left, imgpoints_right, mtx0, dist0,
                                                                 mtx1, dist1, (width, height), criteria = criteria, flags = stereocalibration_flags)

    cv2.destroyAllWindows()
    return R, T

def get_poi(mtx_1):
    fx = mtx_1[0][0]
    fy = mtx_1[1][1]
    cx = mtx_1[0][2]
    cy = mtx_1[1][2]
    return fx,fy,cx,cy

In [7]:
objpoints_1,imgpoints_1,gray_1 = draw_chess_write('1')
objpoints_2,imgpoints_2,gray_2 = draw_chess_write('2')

In [8]:
ret_1, mtx_1, dist_1, rvecs_1, tvecs_1 = cv2.calibrateCamera(objpoints_1, imgpoints_1, gray_1.shape[::-1], None, None)
ret_2, mtx_2, dist_2, rvecs_2, tvecs_2 = cv2.calibrateCamera(objpoints_2, imgpoints_2, gray_2.shape[::-1], None, None)

In [ ]:
frames_prefix_c0 = os.path.join('camera_1_2_frames','image_1*')
frames_prefix_c1 = os.path.join('camera_1_2_frames','image_2*')
R,T = stereo_calibrate(mtx_1,dist_1,mtx_2,dist_2,frames_prefix_c0,frames_prefix_c1)

In [87]:
poi_1 = get_poi(mtx_1)
poi_2 = get_poi(mtx_2)

In [88]:
poi_1

(999.6184685265891, 997.587932339299, 217.2324828046475, 223.21015049905702)

In [89]:
poi_2

(1007.8357024349465, 1005.46347350523, 272.03062315393083, 254.10464615971662)

# Obtenção dos POR

In [90]:
width = cv2.imread('./camera1_images/image_1_1.jpg').shape[1]
height = cv2.imread('./camera1_images/image_1_1.jpg').shape[0]

In [101]:
boresight = R
lever_arm = T